# Decision Tree using ID3

## Import Statement

In [1]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log
import random
from matplotlib import pyplot as plt

## Data Handling

In [8]:
dataset = {'Cloud':['sun','sun','covered','rain','rain','rain','covered','sun','sun','rain','sun','covered','covered'],
       'Temperature':['hot','hot','hot','good','fresh','fresh','fresh','good','fresh','good','good','good','hot'],
       'Humidity':['high','high','high','high','normal','normal','normal','high','normal','normal','normal','high','normal'],
       'Wind':['false','true','false','false','false','true','true','false','false','false','true','true','false'],
       'Golf':['No','No','Yes','Yes','Yes','No','Yes','No','Yes','Yes','Yes','Yes','Yes']}
test_dataset={'Cloud':['rain'],
             'Temperature':['good'],
             'Humidity':['high'],
             'Wind':['true']}

In [9]:
#creating pandas data frames from data
df = pd.DataFrame(dataset,columns=['Cloud','Temperature','Humidity','Wind','Golf'])
testseries=pd.DataFrame(test_dataset)
print(df)

      Cloud Temperature Humidity   Wind Golf
0       sun         hot     high  false   No
1       sun         hot     high   true   No
2   covered         hot     high  false  Yes
3      rain        good     high  false  Yes
4      rain       fresh   normal  false  Yes
5      rain       fresh   normal   true   No
6   covered       fresh   normal   true  Yes
7       sun        good     high  false   No
8       sun       fresh   normal  false  Yes
9      rain        good   normal  false  Yes
10      sun        good   normal   true  Yes
11  covered        good     high   true  Yes
12  covered         hot   normal  false  Yes


## Find Entropy

In [32]:
#function to find entropy of entire dataset
def total_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, adjustable target varable class name
    entropy = 0
    vals = df[Class].unique()
    for val in vals:
        coef = df[Class].value_counts()[val]/len(df[Class])
        entropy += -coef*np.log2(coef)
    return entropy

## Find Entropy for Attribute

In [49]:
#function to find the entropy of an attribute. Used when tying to find the attribute which contribtes to the greatest information gain
def entropy_attribute(df,attribute):
  Class = df.keys()[-1]   #To make the code generic, adjustable target variable class name
  target_vars = df[Class].unique()  #This gives all 'Yes' and 'No'
  vars = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
  entropy2 = 0
  for var in vars:
      entropy = 0
      for target_var in target_vars:
          num = len(df[attribute][df[attribute]==var][df[Class] ==target_var])
          den = len(df[attribute][df[attribute]==var])
          fraction = num/(den+eps)
          entropy += -fraction*log(fraction+eps)
      fraction2 = den/len(df)
      entropy2 += -fraction2*entropy  #Sums up all the entropy E(attribute)
  return abs(entropy2)



## Find Winner (Max Gain)

In [50]:
#Function to find the attribue which contribtes to the greatest inforation gain when creating a new node
def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
        IG.append(total_entropy(df)-entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]

In [51]:
#Functio to create a subtable of a given attriute for specific values of another. i.e all The temperature values for high Humidity
def get_subtable(df, node,val):
  return df[df[node] == val].reset_index(drop=True)

## Function for Building Tree

In [52]:
#Function to actually build the decision tree
def buildTree(df,tree=None): 
    Class = df.keys()[-1]   

    #Get attribute with maximum information gain for root node
    node = find_winner(df)
    
    #Get distinct value of that attribute e.g Temperature is node and good, hot and fresh are values
    attVal = np.unique(df[node])
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
   #make loop to construct a tree by calling this function recursively. 
    #check if the subset is pure and stops if it is pure. 

    for val in attVal:
        
        subtable = get_subtable(df,node,val)
        clVal,counts = np.unique(subtable['Golf'],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][val] = clVal[0]                                                    
        else:        
            tree[node][val] = buildTree(subtable) #Calling the function recursively 
                   
    return tree

## Calling buildtree

In [53]:
#creating an instance 'tree' using the original data set
tree=buildTree(df)

#visualizing tree
import pprint
pprint.pprint(tree)

{'Cloud': {'covered': 'Yes',
           'rain': {'Wind': {'false': 'Yes', 'true': 'No'}},
           'sun': {'Humidity': {'high': 'No', 'normal': 'Yes'}}}}


## Predict Function for Future Input

In [54]:
#function that uses a given tree to make a prediciton based on new data (takes a single series as input)
def predict(inst,tree):
    #This function is used to predict for any input variable 
    
    #Recursively we go through the tree

    for nodes in tree.keys():        
        
        val = inst[nodes]
        #Depeding on the data used to build the tree, some trees may not have the same value range as input data. I this case, the predict function randoml selects one of the values the tree as available
        try:
            tree = tree[nodes][val]
        except KeyError:
            pred=random.choice(list(list(tree.values())[0].values()))
            break
        else:    
            pred = 0
            
        if type(tree) is dict:
            pred = predict(inst, tree)
        else:
            pred= tree
            break;                            
        
    return pred

## Predicting Test Data Instance

In [58]:
#Note that this test data instance will be used for prediction in bagging and random forest applications to come
inst=testseries.iloc[0]
print('The new input data is given as:\n' ,inst)
print('--------------------')
print('\nThe tree used to determine whether or not golf is appropriate:\n', tree)
print('----------------------------------------------')

#calling the predict function on test data
pred=predict(inst,tree)
print('\nThe tree decides '+pred+' to Golf!\n')
print('--------------------')



The new input data is given as:
 Cloud          rain
Temperature    good
Humidity       high
Wind           true
Name: 0, dtype: object
--------------------

The tree used to determine whether or not golf is appropriate:
 {'Cloud': {'covered': 'Yes', 'rain': {'Wind': {'false': 'Yes', 'true': 'No'}}, 'sun': {'Humidity': {'high': 'No', 'normal': 'Yes'}}}}
----------------------------------------------

The tree decides No to Golf!

--------------------


# Bagging

In [59]:
from collections import defaultdict
#number of subsets

def bag(dataframe,subnumber):
    subno=subnumber
    #list of voes by individual trees
    votes=[]
    votes1=[]
    #creating subets to build tree, first taking rows of df then building a dataframe ndf from rows
    for i in range(0,subno):
        indxs=[]
        for i in range(0,len(dataframe)):
            indx = random.randint(0,len(dataframe)-1)
            indxs.append(indx)
        
        subset=defaultdict(list)
        l=[]
        for indx in indxs:
            l.append(df.iloc[indx].to_dict())
        for d in l: 
            for key, value in d.items():
                subset[key].append(value)
        
        #creating a data frame from new subset
        ndf = pd.DataFrame(subset,columns=['Cloud','Temperature','Humidity','Wind','Golf'])
        #building new tree from new data subset and obtaining prediction from test data
        Tree=buildTree(ndf)
        votes.append(predict(inst,Tree))
    #output
    print(str(votes.count('Yes'))+' trees voted to play golf and '+str(votes.count('No'))+' trees voted not to play.')
    if votes.count('Yes') > votes.count('No'): 
        print('Given the knowledge of trees, take to the greens and play golf good fellow!') 
    if votes.count('Yes') < votes.count('No'):
        print('The trees whisper no golf today. Go home and have some tea')
    if votes.count('Yes') == votes.count('No'):
        print('The trees cant decide. Try again or replace the number of subsets with an odd number')


In [60]:
#Calling Bagging function. Trains on random subsets of initial data, is tested on the same test data
bag(df,51)

34 trees voted to play golf and 16 trees voted not to play.
Given the knowledge of trees, take to the greens and play golf good fellow!


## Bagging Function for Random Forest: Does not print output

In [61]:
#Bagging function to be used in the Random Forest function. The only difference between this and the previous bagging function is this returns simply 'Yes' or 'No'
def bagRF(dataframe):
    #creating subets to build tree, first taking rows of df then building a dataframe ndf from rows
    indxs=[]
    for i in range(0,len(dataframe)):
        indx = random.randint(0,len(dataframe)-1)
        indxs.append(indx)
        
    subset=defaultdict(list)
    l=[]
    for indx in indxs:
        l.append(df.iloc[indx].to_dict())
    for d in l: 
        for key, value in d.items():
            subset[key].append(value)
  
    ndf = pd.DataFrame(subset,columns=['Cloud','Temperature','Humidity','Wind','Golf'])
    Tree=buildTree(ndf)
        
    return str(predict(inst,Tree))

# Random Forest

In [47]:
#creating Random Forest function. 
def RF(attributenumb,subnumb,dataframe):
    votes=[]
    for i in range(0,subnumb):
        #Dropping attributes from dataframe
        if attributenumb>=len(dataframe.columns):
            print("try with less attributes")
            return
        if attributenumb<=0:
            print("try with more attributes, and don't be silly")
            return            
        else:
            k=attributenumb
            ndf=dataframe
            l=len(ndf.columns)-1
            m=l-k
            indxs=[]
            indxs=random.sample(range(0,l),m)
            p=ndf.drop(ndf.columns[indxs],axis=1)
            
    #Creating subset with replacement
            votes.append(bagRF(p))
    print(str(votes.count('Yes'))+' trees voted to play golf and '+str(votes.count('No'))+' trees voted not to play.')
    if votes.count('Yes') > votes.count('No'): 
        print('Given the knowledge of trees, take to the greens and play golf good fellow!') 
    if votes.count('Yes') < votes.count('No'):
        print('The trees whisper no golf today. Go home and have some tea')
    if votes.count('Yes') == votes.count('No'):
        print('The trees cant decide. Try again or replace the number of subsets with an odd number')

In [48]:
#calling Random Forest
RF(4,51,df)

36 trees voted to play golf and 15 trees voted not to play.
Given the knowledge of trees, take to the greens and play golf good fellow!
